# Preprocessing

LOOK AT THE PREPROCESSING FILE

In [1]:
from pathlib import Path
import matplotlib.pyplot as plt
from PIL import Image
from shapely.wkt import loads
import json
import os
import pandas as pd
import rasterio as rio
from rasterio.features import rasterize
import geopandas as gpd
import numpy as np
from shapely import wkt
from shapely import Polygon
import numpy as np
import chardet

In [2]:
# HPC Terrabyte
# adapt the user to your needs
USER = "di97ren"
# keep the following unchanged
ROOT = Path("/dss/dsstbyfs02/pn49ci/pn49ci-dss-0022")
USER_PATH = ROOT / f"users/{USER}"
DATA_PATH = ROOT / "data"



# Configure the path to the xView2 dataset for your environment
DATASET_ROOT = DATA_PATH / "xview2-subset"
# noch für hold und tier 3 und test
Train_split1 = DATASET_ROOT / "train"

#IMAGE_FOLDER = Train_split1 / "images/"

#LABEL_FOLDER = Train_split1 / "labels/"

TARGET_FOLDER = Train_split1 / "targets/"

PNG_FOLDER = Train_split1 / "png_images/"



In [4]:
print(os.listdir(TARGET_FOLDER))

['hurricane-michael_00000386_pre_disaster.png', 'socal-fire_00001223_pre_disaster.png', 'midwest-flooding_00000367_post_disaster.png', 'hurricane-harvey_00000118_post_disaster.png', 'socal-fire_00000971_pre_disaster.png', 'hurricane-michael_00000260_post_disaster.png', 'palu-tsunami_00000072_post_disaster.png', 'socal-fire_00000520_pre_disaster.png', 'socal-fire_00001223_post_disaster.png', 'palu-tsunami_00000090_post_disaster.png', 'socal-fire_00000411_post_disaster.png', 'socal-fire_00001349_pre_disaster.png', 'midwest-flooding_00000269_post_disaster.png', 'hurricane-michael_00000527_post_disaster.png', 'hurricane-michael_00000371_pre_disaster.png', 'hurricane-matthew_00000364_pre_disaster.png', 'palu-tsunami_00000154_post_disaster.png', 'hurricane-harvey_00000109_pre_disaster.png', 'hurricane-matthew_00000360_pre_disaster.png', 'socal-fire_00000308_pre_disaster.png', 'palu-tsunami_00000154_pre_disaster.png', 'socal-fire_00000655_pre_disaster.png', 'hurricane-matthew_00000173_post_di

In [10]:

labels = os.listdir(LABEL_FOLDER)


label_paths = []


for l in labels:
    
    label_paths.append(os.path.join(LABEL_FOLDER / l))


label_data = []

for label in label_paths:
    print(label)
    with open(label, "r") as file:
        label_data.append(pd.read_json(file))



/dss/dsstbyfs02/pn49ci/pn49ci-dss-0022/data/xview2/tier1/labels/hurricane-michael_00000170_post_disaster.json
/dss/dsstbyfs02/pn49ci/pn49ci-dss-0022/data/xview2/tier1/labels/socal-fire_00000822_post_disaster.json
/dss/dsstbyfs02/pn49ci/pn49ci-dss-0022/data/xview2/tier1/labels/midwest-flooding_00000411_pre_disaster.json
/dss/dsstbyfs02/pn49ci/pn49ci-dss-0022/data/xview2/tier1/labels/hurricane-florence_00000251_post_disaster.json
/dss/dsstbyfs02/pn49ci/pn49ci-dss-0022/data/xview2/tier1/labels/socal-fire_00001298_post_disaster.json
/dss/dsstbyfs02/pn49ci/pn49ci-dss-0022/data/xview2/tier1/labels/socal-fire_00000845_post_disaster.json
/dss/dsstbyfs02/pn49ci/pn49ci-dss-0022/data/xview2/tier1/labels/socal-fire_00000108_pre_disaster.json
/dss/dsstbyfs02/pn49ci/pn49ci-dss-0022/data/xview2/tier1/labels/hurricane-michael_00000388_pre_disaster.json
/dss/dsstbyfs02/pn49ci/pn49ci-dss-0022/data/xview2/tier1/labels/santa-rosa-wildfire_00000361_post_disaster.json
/dss/dsstbyfs02/pn49ci/pn49ci-dss-0022/

In [11]:
def extract_features(features):
    """Extract polygons, feature types, and damage classes from features."""
    return {
        'geometries': [feature['wkt'] for feature in features],
        'class_name': [feature['properties'].get('feature_type', 'unknown') for feature in features],
        'damage_class': [feature['properties'].get('subtype', 'no-damage') for feature in features]
    }

def load_label_data(label_paths):
    """Load label data from the specified paths."""
    label_data = []
    for label in label_paths:
        with open(label, "r") as file:
            label_data.append(pd.read_json(file))
    return label_data

def process_label_metadata(label):
     """Process metadata from the label."""
     metadata = label['metadata']
     return {
        'img_name': metadata['img_name'][:-4],  # Remove file extension
        'disaster': metadata['disaster'],
        'disaster_type': metadata['disaster_type']
    }

def process_features(label, damage_codes):
     """Process features from the label and apply damage codes."""
     
     feature_data = label['features']['xy']
     feature_dict = extract_features(feature_data)
    
     df = pd.DataFrame(feature_dict)
    
    # Add metadata columns to the dataframe
     metadata = process_label_metadata(label)
     for key, value in metadata.items():
        df[key] = value

    # Apply damage codes
     df['damage_code'] = df['damage_class'].apply(lambda x: damage_codes.get(x, 999))

    # Convert damage codes to integers
     df['damage_code'] = df['damage_code'].astype(int)

     return df

def make_label_dictionary(input_directory, damage_codes):
    """Create a dictionary of labels with associated metadata and damage codes."""
    label_paths = [os.path.join(input_directory, f) for f in os.listdir(input_directory)]
    label_data = load_label_data(label_paths)
    
    label_dictionary = {}
    
    for label in label_data:
        img_name = label['metadata']['img_name'][:-4]  # Remove file extension
        label_df = process_features(label, damage_codes)
        
        # Add the processed dataframe to the dictionary
        label_dictionary[img_name] = label_df

    return label_dictionary

In [12]:
damage_codes = {
    'no-damage' : 1,
    'minor-damage' : 2,
    'major-damage' : 3,
    'destroyed' : 4,
    'un-classified' : 5
}


In [13]:
label_dictionary = make_label_dictionary(LABEL_FOLDER, damage_codes)

In [14]:
def geotiff_converter(image_directoy: dir , output_directory: dir):

    '''
    This function takes the input geotiff images and converts them to png images 
    '''

    images = os.listdir(image_directoy) # get all image names 

        
    # Check if the directory exists
    if not os.path.exists(output_directory):
        # Create the directory if it doesn't exist
        os.makedirs(output_directory)
        print(f"Directory '{output_directory}' created.")
    else:
        print(f"Directory '{output_directory}' already exists.")


    for i in images: # iterate over each image and open it with rasterio

        png_name = i[:-4] + ".png"

        with rio.open( image_directoy / i) as src:
            r , g , b = src.read(1), src.read(2), src.read(3)

            img = np.stack([r, g, b], axis = -1) # Stack the bands to create and np image array 

            # normalize image values:
            if img.dtype != np.uint8:
                img = ((img - img.min()) / (img.max() - img.min()) * 255).astype(np.uint8)

            png_image = Image.fromarray(img) # make it an image

            png_image.save( output_directory / png_name) # save the image


In [15]:
# Apply geotiff_converter to all input images:
geotiff_converter(IMAGE_FOLDER, PNG_FOLDER)

Directory '/dss/dsstbyfs02/pn49ci/pn49ci-dss-0022/data/xview2/tier1/png_images' already exists.


In [16]:
def create_disaster_targets (png_image_directory: dir,
                             label_dictionary: dict, 
                             target_output_directory: dir):
    
    
    # Check if the directory exists
    if not os.path.exists(target_output_directory):
        # Create the directory if it doesn't exist
        os.makedirs(target_output_directory)
        print(f"Directory '{target_output_directory}' created.")
    else:
        print(f"Directory '{target_output_directory}' already exists.")

    
    pngs = os.listdir(png_image_directory)


    for image_name in pngs:

        if "pre_disaster" in image_name:
            label = label_dictionary[image_name[:-4]]['geometries'] # retrieving geometries from the label
            gdf = gpd.GeoDataFrame(geometry=label.apply(wkt.loads)) # creating a geodataframe


            image = Image.open(png_image_directory / image_name) # open the corresponding image

            width,height = image.size # getting width and height information

            # Erstelle eine leere Maske (0 = Hintergrund, 1 = Gebäude/Label)
            mask = np.zeros((height, width), dtype=np.uint8)

            # Rasterisiere die Polygone in die Maske
            shapes = [(geom, 1) for geom in gdf.geometry]  # Alle Polygone mit Wert 1 versehen
            mask = rio.features.rasterize(shapes, out_shape=(height, width))
            mask_img = Image.fromarray(mask.astype(np.uint8))
            #mask_img = Image.fromarray(mask * 255)  # Skaliere 0/1 auf 0/255 für Darstellung
            mask_img.save(target_output_directory / image_name )

        else:
            label = label_dictionary[image_name[:-4]]
            gdf_post = gpd.GeoDataFrame({
                'geometry': [wkt.loads(wkt_string) for wkt_string in label['geometries']],
                'damage_code': label['damage_code']
            })

            image = Image.open(png_image_directory / image_name) # open the corresponding image

            width,height = image.size # getting width and height information

            # Erstelle eine leere Maske (0 = Hintergrund, 1 = Gebäude/Label)
            mask = np.zeros((height, width), dtype=np.uint8)

            # Rasterisiere die Polygone in die Maske
            shapes = [(geom, damage) for geom, damage in zip(gdf_post.geometry, gdf_post.damage_code)]
            mask = rio.features.rasterize(shapes, out_shape=(height, width))

            mask_img = Image.fromarray(mask.astype(np.uint8))

            mask_img.save(target_output_directory / image_name)



In [17]:
create_disaster_targets(PNG_FOLDER, label_dictionary, TARGET_FOLDER)

Directory '/dss/dsstbyfs02/pn49ci/pn49ci-dss-0022/data/xview2/tier1/targets' already exists.


# DatasetClass and DataLoader

In [3]:
from typing import Tuple, Dict, List
from pathlib import Path
import numpy as np
# HPC Terrabyte
# adapt the user to your needs
USER = "di97ren"
# keep the following unchanged
ROOT = Path("/dss/dsstbyfs02/pn49ci/pn49ci-dss-0022")
USER_PATH = ROOT / f"users/{USER}"
DATA_PATH = ROOT / "data"


# Configure the path to the xview2 dataset for your environment
DATASET_ROOT = DATA_PATH / "xview2"
TRAIN_ROOT = DATASET_ROOT / "tier1"
TRAIN_IMG = TRAIN_ROOT / "png_images"
TRAIN_TARGET = TRAIN_ROOT / 'targets'

VAL_ROOT = DATASET_ROOT / "hold"
VAL_IMG = TRAIN_ROOT / "png_images"
VAL_TARGET = TRAIN_ROOT / 'targets'



In [2]:
import os

def iterate_through_dir(directory_path):
    for directory_path, directorynames, filenames in os.walk(directory_path):
        print(f"There are {len(directorynames)} directories and {len(filenames)} images in '{directory_path}'.")

In [3]:
iterate_through_dir(DATASET_ROOT)

There are 4 directories and 2 images in '/dss/dsstbyfs02/pn49ci/pn49ci-dss-0022/data/xview2'.
There are 4 directories and 0 images in '/dss/dsstbyfs02/pn49ci/pn49ci-dss-0022/data/xview2/tier3'.
There are 0 directories and 12738 images in '/dss/dsstbyfs02/pn49ci/pn49ci-dss-0022/data/xview2/tier3/labels'.
There are 0 directories and 12738 images in '/dss/dsstbyfs02/pn49ci/pn49ci-dss-0022/data/xview2/tier3/images'.
There are 0 directories and 12738 images in '/dss/dsstbyfs02/pn49ci/pn49ci-dss-0022/data/xview2/tier3/targets'.
There are 0 directories and 12738 images in '/dss/dsstbyfs02/pn49ci/pn49ci-dss-0022/data/xview2/tier3/png_images'.
There are 4 directories and 0 images in '/dss/dsstbyfs02/pn49ci/pn49ci-dss-0022/data/xview2/hold'.
There are 0 directories and 1866 images in '/dss/dsstbyfs02/pn49ci/pn49ci-dss-0022/data/xview2/hold/labels'.
There are 0 directories and 1866 images in '/dss/dsstbyfs02/pn49ci/pn49ci-dss-0022/data/xview2/hold/images'.
There are 0 directories and 1866 images 

In [4]:
TRAIN_ROOT = DATASET_ROOT
TEST_ROOT = DATASET_ROOT

TRAIN_ROOT, TEST_ROOT

(PosixPath('/dss/dsstbyfs02/pn49ci/pn49ci-dss-0022/data/xview2'),
 PosixPath('/dss/dsstbyfs02/pn49ci/pn49ci-dss-0022/data/xview2'))

In [4]:
import torch
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, transforms
import torchvision.transforms.v2 as v2
import matplotlib.pyplot as plt
from PIL import Image

In [6]:
# def transform():
#     """Transform für Bilder & Masken"""
#     return v2.Compose([
#         v2.RandomHorizontalFlip(),
#         v2.RandomVerticalFlip(),
#         v2.RandomRotation(degrees=15),
#         v2.RandomAffine(degrees=0, translate=(0.05, 0.05), scale=(0.95, 1.05)),
#         v2.ToDtype(torch.float32, scale=True)  # Automatische Skalierung auf [0,1]
#     ])

# def image_transform():
#     """Nur für RGB-Bilder"""
#     return v2.Compose([
#         v2.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
#         v2.GaussianBlur(kernel_size=(3, 3), sigma=(0.1, 2.0)),
#         v2.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
#     ])

In [7]:
# class xView2Dataset(Dataset):

#     def __init__(self,
#                  png_path: str,
#                  target_path: str,
#                  transform: callable = None,
#                  image_transform: callable = None):
        
#         self.png_path = png_path
#         self.target_path = target_path
#         self.transform = transform
#         self.image_transform = image_transform
        

#         # get all pre-disaster images:
#         self.pre_images = sorted(self.png_path.glob("*_pre_disaster.png"))
        
#         self.pairs = [] #

#         for pre_img_path in self.pre_images:
#             post_img_path = self.png_path / pre_img_path.name.replace("_pre_disaster", "_post_disaster")

#             post_target_path = self.target_path / pre_img_path.name.replace("_pre_disaster", "_post_disaster")
#             pre_target_path = pre_target_path = self.target_path / pre_img_path.name


#             if post_img_path.exists() and post_target_path.exists() and pre_target_path.exists():
#                 self.pairs.append((pre_img_path, post_img_path, pre_target_path, post_target_path))

#         assert len(self.pairs) > 0, "No matching image-pairs found!"

#         # super().__init__()

#     def __len__(self):
#         return len(self.pre_images)

#     def __getitem__(self, index):
#         pre_img_path, post_img_path, pre_target_path, post_target_path = self.pairs[index]

#         # load images and target masks with 
        
#         pre_img = Image.open(pre_img_path).convert("RGB")
#         post_img = Image.open(post_img_path).convert("RGB")
#         pre_target_mask = Image.open(pre_target_path).convert('L')
#         post_target_mask = Image.open(post_target_path).convert('L')

#         # convert to numpy arrays
#         pre_img = np.array(pre_img, dtype=np.float32) / 255.0
#         post_img = np.array(post_img, dtype=np.float32) / 255.0
#         pre_target_mask = np.array(pre_target_mask, dtype=np.float32)
#         post_target_mask = np.array(post_target_mask, dtype=np.float32)

#         # convert to Tensor
#         pre_img = torch.tensor(pre_img).permute(2, 0, 1)  # (H, W, C) → (C, H, W)
#         post_img = torch.tensor(post_img).permute(2, 0, 1)
#         pre_target_mask = torch.tensor(pre_target_mask).unsqueeze(0)  # (H, W) → (1, H, W)
#         post_target_mask = torch.tensor(post_target_mask).unsqueeze(0)

#     # Transformation (optional)


#         if self.transform:
#             stack = torch.cat([pre_img, post_img, pre_target_mask, post_target_mask], dim=0)  # (8, H, W)
#             stack = self.transform(stack)

#             pre_img, post_img, pre_target_mask, post_target_mask = stack[:3], stack[3:6], stack[6:7], stack[7:8]
        
#         if self.image_transform:
            
#             # Nur auf Bilder Normalisierung anwenden
#             pre_img = self.image_transform(pre_img)
#             post_img = self.image_transform(post_img)

#         return pre_img, post_img, pre_target_mask, post_target_mask 
    

    
        

In [5]:
def transform():
    """Transform für Bilder & Masken"""
    return v2.Compose([
        v2.RandomHorizontalFlip(),
        v2.RandomVerticalFlip(),
        v2.RandomRotation(degrees=15),
        v2.RandomAffine(degrees=0, translate=(0.05, 0.05), scale=(0.95, 1.05)),
        v2.ToDtype(torch.float32, scale=True)  # Automatische Skalierung auf [0,1]
    ])

def image_transform():
    """Nur für RGB-Bilder"""
    return v2.Compose([
        v2.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
        v2.GaussianBlur(kernel_size=(3, 3), sigma=(0.1, 2.0)),
        v2.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

class xView2Dataset(Dataset):

    def __init__(self,
                 png_path: str,
                 target_path: str,
                 transform: callable = None,
                 image_transform: callable = None):
        
        self.png_path = png_path
        self.target_path = target_path
        self.transform = transform
        self.image_transform = image_transform
        

        # get all pre-disaster images:
        self.pre_images = sorted(self.png_path.glob("*_pre_disaster.png"))
        
        self.pairs = [] #

        for pre_img_path in self.pre_images:
            post_img_path = self.png_path / pre_img_path.name.replace("_pre_disaster", "_post_disaster")

            post_target_path = self.target_path / pre_img_path.name.replace("_pre_disaster", "_post_disaster")
            pre_target_path = pre_target_path = self.target_path / pre_img_path.name


            if post_img_path.exists() and post_target_path.exists() and pre_target_path.exists():
                self.pairs.append((pre_img_path, post_img_path, pre_target_path, post_target_path))

        assert len(self.pairs) > 0, "No matching image-pairs found!"

        # super().__init__()

    def __len__(self):
        return len(self.pre_images)

    def __getitem__(self, index):
        pre_img_path, post_img_path, pre_target_path, post_target_path = self.pairs[index]

        # load images and target masks with 
        
        pre_img = Image.open(pre_img_path).convert("RGB")
        post_img = Image.open(post_img_path).convert("RGB")
        pre_target_mask = Image.open(pre_target_path).convert('L')
        post_target_mask = Image.open(post_target_path).convert('L')

        # convert to numpy arrays
        pre_img = np.array(pre_img, dtype=np.float32) / 255.0
        post_img = np.array(post_img, dtype=np.float32) / 255.0
        pre_target_mask = np.array(pre_target_mask, dtype=np.float32)
        post_target_mask = np.array(post_target_mask, dtype=np.float32)

        # convert to Tensor
        pre_img = torch.tensor(pre_img).permute(2, 0, 1)  # (H, W, C) → (C, H, W)
        post_img = torch.tensor(post_img).permute(2, 0, 1)
        pre_target_mask = torch.tensor(pre_target_mask).unsqueeze(0)  # (H, W) → (1, H, W)
        post_target_mask = torch.tensor(post_target_mask).unsqueeze(0)

    # Transformation (optional)


        if self.transform:
            stack = torch.cat([pre_img, post_img, pre_target_mask, post_target_mask], dim=0)  # (8, H, W)
            stack = self.transform(stack)

            pre_img, post_img, pre_target_mask, post_target_mask = stack[:3], stack[3:6], stack[6:7], stack[7:8]
        
        if self.image_transform:
            
            # Nur auf Bilder Normalisierung anwenden
            pre_img = self.image_transform(pre_img)
            post_img = self.image_transform(post_img)

        return pre_img, post_img, pre_target_mask, post_target_mask 
    

    
        
        

In [6]:
def collate_fn(batch):
      # Extrahieren der einzelnen Daten aus der Batch-Liste
    pre_imgs, post_imgs, pre_masks, post_masks = zip(*batch)

    # Stapeln der Tensoren entlang der Batch-Dimension (erste Dimension)
    pre_imgs = torch.stack(pre_imgs, dim=0)
    post_imgs = torch.stack(post_imgs, dim=0)
    pre_masks = torch.stack(pre_masks, dim=0)
    post_masks = torch.stack(post_masks, dim=0)

    return pre_imgs, post_imgs, pre_masks, post_masks


In [7]:
train_dataset = xView2Dataset(png_path = TRAIN_IMG,
                 target_path = TRAIN_TARGET,
                 transform = transform(),
                 image_transform = image_transform()
                 )

val_dataset = xView2Dataset(png_path = VAL_IMG ,
                 target_path = VAL_TARGET,
                 transform = transform(),
                 image_transform = image_transform()
                 )

train_dataset, val_dataset
len(train_dataset), len(val_dataset)

(2798, 2798)

# Model

In [8]:
import torchvision.models as models
from torchinfo import summary
import torch.nn as nn


## U-Net Template

In [9]:
import torch
import torch.nn as nn
import torchvision.models as models
import torch.nn.functional as F

class UNet_ResNet50(nn.Module):
    def __init__(self, n_class):
        super().__init__()
        
        # Load ResNet50 als Encoder
        resnet = models.resnet50(weights='ResNet50_Weights.DEFAULT')
        
        # Encoder-Pfad
        self.encoder = nn.Sequential(
            resnet.conv1,
            resnet.bn1,
            resnet.relu,
            resnet.maxpool,
            resnet.layer1,
            resnet.layer2,
            resnet.layer3,
            resnet.layer4
        )
        
        # Decoder mit progressivem Upsampling
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(2048, 1024, kernel_size=2, stride=2),
            nn.ReLU(),
            nn.ConvTranspose2d(1024, 512, kernel_size=2, stride=2),
            nn.ReLU(),
            nn.ConvTranspose2d(512, 256, kernel_size=2, stride=2),
            nn.ReLU(),
            nn.ConvTranspose2d(256, 64, kernel_size=2, stride=2),
            nn.ReLU()
        )
        
        # Finale Klassifikationsschicht
        self.final_conv = nn.Conv2d(64, n_class, kernel_size=1)

    def forward(self, x):
        # Speichere Ursprungsgröße
        original_size = x.shape[2:]
        
        try:
            # Encoder-Durchgang
            features = self.encoder(x)
            #print(f"Encoder Output Shape: {features.shape}")

            # Decoder-Durchgang
            decoder_out = self.decoder(features)
            #print(f"Decoder Output Shape: {decoder_out.shape}")

            # Finale Konvolution
            out = self.final_conv(decoder_out)
            #print(f"Final Conv Output Shape: {out.shape}")

            # Upsample auf Originalgröße
            out = F.interpolate(out, size=original_size, mode='bilinear', align_corners=False)
            #print(f"Final Interpolated Output Shape: {out.shape}")

            return out

        except Exception as e:
            print(f"Error in forward pass: {e}")
            raise

# # Test-Funktion mit Fehlerbehandlung
# def test_model():
#     try:
#         input_img = torch.rand(5, 3, 1024, 1024)
#         model = UNet_ResNet50(n_class=5)
#         output = model(input_img)
#         print("Successful Model Test")
#         print("Input shape:", input_img.shape)
#         print("Output shape:", output.shape)
#     except Exception as e:
#         print(f"Test failed: {e}")

# Ausführen des Tests


## Siamese Network

In [10]:

class SiameseUnet(nn.Module):
    def __init__(self, num_pre_classes=2, num_post_classes=6):
        super(SiameseUnet, self).__init__()

        self.unet_preDisaster = UNet_ResNet50(n_class=num_pre_classes)
        self.unet_postDisaster = UNet_ResNet50(n_class=num_post_classes)

        # Fusion-Layer kombiniert prä- und post-Klassifikationen
        self.fusion_layer = nn.Sequential(
            nn.Conv2d(num_pre_classes + num_post_classes, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(64, num_pre_classes + num_post_classes, kernel_size=1)
        )

    def forward(self, pre_image, post_image):
        pre_output = self.unet_preDisaster(pre_image)
        post_output = self.unet_postDisaster(post_image)

        # Konkatenieren der Ausgaben
        fused_output = torch.cat([pre_output, post_output], dim=1)

        # Fusion der Features
        final_output = self.fusion_layer(fused_output)

        return final_output

# Training and Validation

In [11]:
device = "cuda" if torch.cuda.is_available() else "cpu"

if device == "cuda":
    num_workers = torch.cuda.device_count() * 4



In [12]:

train_dataloader = DataLoader(
    train_dataset,
    batch_size= 32, 
    collate_fn = collate_fn,
    shuffle= True,
    num_workers = num_workers,
    prefetch_factor=2, # number of preloaded batches for each workre
    drop_last= True
)

val_dataloader =  DataLoader(
    val_dataset,
    batch_size= 32, 
    collate_fn = collate_fn,
    shuffle= True,
    num_workers = num_workers,
    prefetch_factor=2, # number of preloaded batches for each workre
    drop_last= True
)

# lerning_rate = batch_size/16

In [ ]:
train_dataloader

In [ ]:
# Beispiel, wie du durch den DataLoader iterierst
for pre_imgs, post_imgs, pre_masks, post_masks in train_dataloader:
    print(f"Pre-Disaster Batch Shape: {pre_imgs.shape}")
    print(f"Post-Disaster Batch Shape: {post_imgs.shape}")
    print(f"Pre-Mask Batch Shape: {pre_masks.shape}")
    print(f"Post-Mask Batch Shape: {post_masks.shape}")
    break

In [13]:
import torch
from torch.optim.lr_scheduler import StepLR, CosineAnnealingLR, ReduceLROnPlateau
from torchmetrics.classification import MulticlassPrecision, MulticlassRecall, MulticlassF1Score
from torch.utils.tensorboard import SummaryWriter
from pathlib import Path

NUM_CLASSES = 6
# PRE_DESASTER_CLASSES = 2
# POST_DESASTER_CLASSES = 6

EPOCHS = 80
EXPERIMENT_GROUP = "xView2"
EXPERIMENT_ID = "UNet_001"

# HPC Terrabyte path config
EXPERIMENT_DIR = USER_PATH / f"experiments/{EXPERIMENT_GROUP}"
EXPERIMENT_DIR.mkdir(parents=True, exist_ok=True)

CHECKPOINTS_DIR = USER_PATH / f"checkpoints/{EXPERIMENT_GROUP}"
CHECKPOINTS_DIR.mkdir(parents=True, exist_ok=True)
model = SiameseUnet()
if torch.cuda.device_count() > 1:
    print(f"Verwende {torch.cuda.device_count()} GPUs!")
    model = torch.nn.DataParallel(model)
model = model.cuda()


writer = SummaryWriter(EXPERIMENT_DIR / EXPERIMENT_ID)

loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.008, momentum=0.9, weight_decay=1e-6)

#scheduler = StepLR(optimizer, step_size=40, gamma=0.1)
scheduler = CosineAnnealingLR(optimizer, T_max=EPOCHS, eta_min=1e-6)

# if you want to use this, see further comments tagged with #ROP
# scheduler = ReduceLROnPlateau(optimizer, mode="min", patience=5, factor=0.1)



Verwende 4 GPUs!


In [ ]:

# Hol den ersten Batch
for batch, (pre_imgs, post_imgs, pre_masks, post_masks) in enumerate(train_dataloader):
    print("Pre masks unique values:", torch.unique(pre_masks))
    print("Post masks unique values:", torch.unique(post_masks))
    print("Pre masks shape:", pre_masks.shape)
    print("Post masks shape:", post_masks.shape)
    print("Pre masks dtype:", pre_masks.dtype)
    print("Post masks dtype:", post_masks.dtype)

    break






In [14]:
def train_step(model, dataloader, loss_fn, optimizer, epoch):
    model.train()
    train_loss = 0.0

    precision_pre = MulticlassPrecision(num_classes=2).to(device)
    recall_pre = MulticlassRecall(num_classes=2).to(device)
    f1_pre = MulticlassF1Score(num_classes=2).to(device)
    
    precision_post = MulticlassPrecision(num_classes=6).to(device)
    recall_post = MulticlassRecall(num_classes=6).to(device)
    f1_post = MulticlassF1Score(num_classes=6).to(device)
    
    for pre_imgs, post_imgs, pre_masks, post_masks in dataloader:
        X_pre = pre_imgs.to(device)
        y_pre = pre_masks.to(device)
        X_post = post_imgs.to(device)
        y_post = post_masks.to(device)

        # Forward pass
        pred = model(X_pre, X_post)

        # Prepare masks      
        y_pre_metric = y_pre.squeeze(1).long()
        y_post_metric = y_post.squeeze(1).long()

        # Loss Berechnung
        # Pre-Bilder: erste 2 Kanäle (Klassen 0,1)
        loss_pre = loss_fn(pred[:, :2], y_pre_metric)
        
        # Post-Bilder: letzte 6 Kanäle (Klassen 0,1,2,3,4,5)
        loss_post = loss_fn(pred[:, 2:], y_post_metric)
        
        loss = loss_pre + loss_post
        
        # Optimizer-Schritte
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Metriken aktualisieren für pre-Bilder
        precision_pre.update(pred[:, :2], y_pre_metric)
        recall_pre.update(pred[:, :2], y_pre_metric)
        f1_pre.update(pred[:, :2], y_pre_metric)

        # Metriken aktualisieren für post-Bilder
        precision_post.update(pred[:, 2:], y_post_metric)
        recall_post.update(pred[:, 2:], y_post_metric)
        f1_post.update(pred[:, 2:], y_post_metric)

        train_loss += loss.item()

    # Metriken berechnen
    precision_pre_value = precision_pre.compute()
    recall_pre_value = recall_pre.compute()
    f1_pre_value = f1_pre.compute()
    
    precision_post_value = precision_post.compute()
    recall_post_value = recall_post.compute()
    f1_post_value = f1_post.compute()

    # TensorBoard Logging
    writer.add_scalar("Loss/Train", train_loss / len(dataloader), epoch)
    
    # Logging für pre-Bilder
    writer.add_scalar("Precision_Pre/Train", precision_pre_value.mean(), epoch)
    writer.add_scalar("Recall_Pre/Train", recall_pre_value.mean(), epoch)
    writer.add_scalar("F1_Score_Pre/Train", f1_pre_value.mean(), epoch)
    
    # Logging für post-Bilder
    writer.add_scalar("Precision_Post/Train", precision_post_value.mean(), epoch)
    writer.add_scalar("Recall_Post/Train", recall_post_value.mean(), epoch)
    writer.add_scalar("F1_Score_Post/Train", f1_post_value.mean(), epoch)

    avg_train_loss = train_loss / len(dataloader) 

    return avg_train_loss

In [15]:
def val_step(model, dataloader, loss_fn, epoch):
    num_batches = len(dataloader)
    model.eval()

    val_loss = 0.0

    precision_pre = MulticlassPrecision(num_classes=2).to(device)
    recall_pre = MulticlassRecall(num_classes=2).to(device)
    f1_pre = MulticlassF1Score(num_classes=2).to(device)
    
    precision_post = MulticlassPrecision(num_classes=6).to(device)
    recall_post = MulticlassRecall(num_classes=6).to(device)
    f1_post = MulticlassF1Score(num_classes=6).to(device)

    with torch.no_grad():
        for pre_imgs, post_imgs, pre_masks, post_masks in dataloader:
            X_pre = pre_imgs.to(device)
            y_pre = pre_masks.to(device)
            X_post = post_imgs.to(device)
            y_post = post_masks.to(device)

            # Forward pass
            pred = model(X_pre, X_post)

            # Prepare masks      
            y_pre_metric = y_pre.squeeze(1).long()
            y_post_metric = y_post.squeeze(1).long()

            # Loss Berechnung
            val_loss_pre = loss_fn(pred[:, :2], y_pre_metric)
            val_loss_post = loss_fn(pred[:, 2:], y_post_metric)
            val_loss += (val_loss_pre.item() + val_loss_post.item())

            # Metriken aktualisieren für pre-Bilder
            precision_pre.update(pred[:, :2], y_pre_metric)
            recall_pre.update(pred[:, :2], y_pre_metric)
            f1_pre.update(pred[:, :2], y_pre_metric)

            # Metriken aktualisieren für post-Bilder
            precision_post.update(pred[:, 2:], y_post_metric)
            recall_post.update(pred[:, 2:], y_post_metric)
            f1_post.update(pred[:, 2:], y_post_metric)

        val_loss /= num_batches

        # Metriken berechnen
        precision_pre_value = precision_pre.compute().cpu().numpy()
        recall_pre_value = recall_pre.compute().cpu().numpy()
        f1_pre_value = f1_pre.compute().cpu().numpy()
        
        precision_post_value = precision_post.compute().cpu().numpy()
        recall_post_value = recall_post.compute().cpu().numpy()
        f1_post_value = f1_post.compute().cpu().numpy()

        # TensorBoard Logging
        writer.add_scalar("Loss/Val", val_loss, epoch)
        
        # Logging für pre-Bilder
        writer.add_scalar("Precision_Pre/Val", precision_pre_value.mean(), epoch)
        writer.add_scalar("Recall_Pre/Val", recall_pre_value.mean(), epoch)
        writer.add_scalar("F1_Score_Pre/Val", f1_pre_value.mean(), epoch)
        
        # Logging für post-Bilder
        writer.add_scalar("Precision_Post/Val", precision_post_value.mean(), epoch)
        writer.add_scalar("Recall_Post/Val", recall_post_value.mean(), epoch)
        writer.add_scalar("F1_Score_Post/Val", f1_post_value.mean(), epoch)

        return val_loss

In [16]:
_ = model.to(device)
best_val_loss = float('inf')
for epoch in range(EPOCHS):
    print(f"Epoch {epoch}")
    avg_train_loss = train_step(model, train_dataloader, loss_fn, optimizer, epoch)
    print(avg_train_loss)
    avg_val_loss = val_step(model, val_dataloader, loss_fn, epoch)
    print(avg_val_loss)
    #ROP: comment the scheduler below and use commeted lines instead
    scheduler.step()
    # scheduler.step(val_loss)
    
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        best_checkpoint_path = CHECKPOINTS_DIR / f'{EXPERIMENT_GROUP}_{EXPERIMENT_ID}_epoch_{epoch}.pth'
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'scheduler_state_dict': scheduler.state_dict(),
            'loss': best_val_loss,
        }, best_checkpoint_path)
        print(f'Checkpoint saved at {best_checkpoint_path}')
        best_checkpoint_path_str = "val_loss:" + str(best_val_loss) + "@" + str(best_checkpoint_path)
        writer.add_text("Best Checkpoint Path", str(best_checkpoint_path_str), epoch)

    writer.add_scalar("LR", scheduler.get_last_lr()[0], epoch)

    # uncomment the following lines if you want to save a checkpoint after each epoch
    # Note: This approach might take a lot of storage
    #torch.save({
    #        'epoch': epoch,
    #        'model_state_dict': model.state_dict(),
    #        'optimizer_state_dict': optimizer.state_dict(),
    #        'scheduler_state_dict': scheduler.state_dict(),
    #        'loss': val_loss,
    #    }, CHECKPOINTS_DIR / f"{EXPERIMENT_ID}_epoch{epoch}.pth")
    
writer.close()
print("Done!")

Epoch 0


0.892661832529923
0.4624404541548641
Checkpoint saved at /dss/dsstbyfs02/pn49ci/pn49ci-dss-0022/users/di97ren/checkpoints/xView2/xView2_UNet_001_epoch_0.pth
Epoch 1
0.45763723124032735
0.45531739945384275
Checkpoint saved at /dss/dsstbyfs02/pn49ci/pn49ci-dss-0022/users/di97ren/checkpoints/xView2/xView2_UNet_001_epoch_1.pth
Epoch 2
0.45524159998729313
0.4529365763582032
Checkpoint saved at /dss/dsstbyfs02/pn49ci/pn49ci-dss-0022/users/di97ren/checkpoints/xView2/xView2_UNet_001_epoch_2.pth
Epoch 3
0.4537238900003762
0.4550342859550454
Epoch 4
0.45349889925156517
0.4547276991879803
Epoch 5
0.4543931378030229
0.45258679018280973
Checkpoint saved at /dss/dsstbyfs02/pn49ci/pn49ci-dss-0022/users/di97ren/checkpoints/xView2/xView2_UNet_001_epoch_5.pth
Epoch 6
0.45434987887568856
0.45336911758814735
Epoch 7
0.4532463338183261
0.4527931688674565
Epoch 8
0.4523538798093796
0.45173354618165684
Checkpoint saved at /dss/dsstbyfs02/pn49ci/pn49ci-dss-0022/users/di97ren/checkpoints/xView2/xView2_UNet_001

# Inference